In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split
from torch_pconv import PConv2d  # Partial convolution layer; ensure this is installed.

In [2]:
class DepthInpaintingDatasetCSV(Dataset):
    def __init__(self, csv_file, target_size=(240,320), mask_size=None, transform=None, apply_mask=True):
        self.data = pd.read_csv(csv_file, header=None)
        self.color_paths = self.data.iloc[:, 0].tolist()
        self.depth_paths = self.data.iloc[:, 1].tolist()
        self.target_size = target_size        # For depth target (and mask): e.g. (240,320)
        self.transform = transform
        self.apply_mask = apply_mask

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Process the RGB image.
        rgb_path = self.color_paths[idx]
        rgb_path = os.path.join("..\\nyu_data\\", rgb_path)  # adjust path if needed
        rgb = cv2.imread(rgb_path)
        if rgb is None:
            raise ValueError(f"RGB image not found: {rgb_path}")
        rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
        rgb = cv2.resize(rgb, (self.target_size[1], self.target_size[0]))
        rgb = cv2.normalize(rgb.astype(np.float32), None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
        rgb = np.clip(rgb, 0, 1)

        # print("RGB:", min(rgb.flatten()), max(rgb.flatten()))  # Debugging line to check RGB values

        # Process the depth image.
        depth_path = self.depth_paths[idx]
        depth_path = os.path.join("..\\nyu_data\\", depth_path)
        depth = cv2.imread(depth_path, cv2.IMREAD_UNCHANGED)
        if depth is None:
            raise ValueError(f"Depth image not found: {depth_path}")
        # Resize depth to target_size (240,320)
        depth = cv2.resize(depth, (self.target_size[1], self.target_size[0]))
        depth = cv2.normalize(depth.astype(np.float32), None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
        depth = np.clip(depth, 0, 1)

        # print("DEPTH:", min(depth.flatten()), max(depth.flatten()))  # Debugging line to check RGB values

        H, W = self.target_size  # e.g., (240,320)
        
        if self.apply_mask:
            mask = self.create_random_mask((H, W))  # 0 for masked, 255 for unmasked.
            masked_img = rgb.copy()
            masked_depth = depth.copy()

            mask_3d = np.stack([mask] * 3, axis=-1)
            
            masked_img[mask_3d == 0] = 1.0 
            masked_depth[mask == 0]  = 1.0    
        
        # Convert to torch tensors.
        img_tensor = torch.from_numpy(rgb).permute(2, 0, 1)                 # (3, H, W)
        masked_img_tensor = torch.from_numpy(masked_img).permute(2, 0, 1)   # (3, H, W)
        masked_depth_tensor = torch.from_numpy(masked_depth).unsqueeze(0)   # (1, H, W)
        
        # Create combined input and target.
        rgb_input = torch.cat([masked_img_tensor], dim=0)    # (3, H, W)
        depth_input = torch.cat([masked_depth_tensor], dim=0)  # (1, H, W)
        target = torch.cat([img_tensor], dim=0)                        # (3, H, W)
        
        if self.transform:
            rgb_input = self.transform(rgb_input)
            depth_input = self.transform(depth_input)
            target = self.transform(target)
            
        return {
            "rgb_input": rgb_input,
            "depth_input": depth_input,
            "target": target,
        }
    
    def create_random_mask(self, size):
        """
        Create a random mask of shape (H, W) using random lines.
        In the resulting mask, pixels with value 255 are unmasked, and pixels with value 0 are masked.
        """
        H, W = size
        mask = np.ones((H, W), dtype=np.uint8) * 255
        num_lines = np.random.randint(5, 10)  # Adjust the number of random lines if desired.
        for _ in range(num_lines):
            x1, x2 = np.random.randint(0, W, size=2)
            y1, y2 = np.random.randint(0, H, size=2)
            thickness = np.random.randint(1, 4)
            cv2.line(mask, (x1, y1), (x2, y2), 0, thickness) 
        return mask

In [3]:
# A simple convolutional block using built-in Sequential.
def conv_block(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
    )

# Encoder module that downsamples the input and returns skip connections.
class Encoder(nn.Module):
    def __init__(self, in_channels, base_channels=8, levels=3):
        super(Encoder, self).__init__()
        self.levels = levels
        self.blocks = nn.ModuleList()
        for i in range(levels):
            out_channels = base_channels * (2 ** i)
            self.blocks.append(conv_block(in_channels, out_channels))
            in_channels = out_channels
        self.pool = nn.MaxPool2d(2)
        
    def forward(self, x):
        skips = []
        for block in self.blocks:
            x = block(x)
            skips.append(x)
            x = self.pool(x)
        return skips, x  # return list of skip features and bottleneck

# Decoder block that upsamples and fuses with the skip connection.
class DecoderBlock(nn.Module):
    def __init__(self, in_channels, skip_channels, out_channels):
        super(DecoderBlock, self).__init__()
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.conv = conv_block(in_channels + skip_channels, out_channels)
        
    def forward(self, x, skip):
        x = self.upsample(x)
        x = torch.cat([x, skip], dim=1)
        x = self.conv(x)
        return x

# Decoder that uses a list of DecoderBlocks.
class Decoder(nn.Module):
    def __init__(self, bottleneck_channels, skip_channels, levels=3):
        super(Decoder, self).__init__()
        self.levels = levels
        self.blocks = nn.ModuleList()
        in_channels = bottleneck_channels
        # skip_channels should be provided in order from shallowest to deepest.
        for i in range(levels):
            # Use skip from the reverse order.
            skip_ch = skip_channels[-(i+1)]
            out_channels = skip_ch // 2
            self.blocks.append(DecoderBlock(in_channels, skip_ch, out_channels))
            in_channels = out_channels
        self.final_conv = nn.Conv2d(in_channels, 3, kernel_size=1)
        
    def forward(self, x, skips):
        for i, block in enumerate(self.blocks):
            skip = skips[-(i+1)]
            x = block(x, skip)
        x = self.final_conv(x)
        return x
    

# A self-attention module using PyTorch's built-in MultiheadAttention.
class MultiheadSelfAttention(nn.Module):
    def __init__(self, in_dim, num_heads=4):
        super(MultiheadSelfAttention, self).__init__()
        # nn.MultiheadAttention expects input shape (B, L, C) if batch_first=True.
        self.mha = nn.MultiheadAttention(embed_dim=in_dim, num_heads=num_heads, batch_first=True)
        
    def forward(self, x):
        # x shape: (B, C, H, W) --> flatten spatial dims to sequence: (B, H*W, C)
        B, C, H, W = x.size()
        x_flat = x.view(B, C, H * W).permute(0, 2, 1)  # now (B, L, C) with L=H*W
        attn_output, _ = self.mha(x_flat, x_flat, x_flat)
        # reshape back to (B, C, H, W)
        attn_output = attn_output.permute(0, 2, 1).view(B, C, H, W)
        return attn_output


# Full model: Dual-branch encoder for RGB and depth, attention fusion, then decoder.
class DepthEnhancedInpaintingNet(nn.Module):
    def __init__(self, base_channels=8, levels=3, num_heads=4):
        super(DepthEnhancedInpaintingNet, self).__init__()
        # Encoders for RGB (3 channels) and depth (1 channel)
        self.rgb_encoder   = Encoder(in_channels=3, base_channels=base_channels, levels=levels)
        self.depth_encoder = Encoder(in_channels=1, base_channels=base_channels, levels=levels)
        
        # Bottleneck channels for each encoder (last block channels)
        bottleneck_rgb   = base_channels * (2 ** (levels - 1))
        bottleneck_depth = base_channels * (2 ** (levels - 1))
        fused_bottleneck_channels = bottleneck_rgb + bottleneck_depth
        
        # Fuse bottleneck features with multihead self-attention
        self.attention = MultiheadSelfAttention(in_dim=fused_bottleneck_channels, num_heads=num_heads)
        
        # Skip channels: each encoder level produces channels = base_channels * (2**i)
        # Fused skip channels from both branches are doubled.
        fused_skips = [base_channels * (2 ** i) * 2 for i in range(levels)]
        self.decoder = Decoder(bottleneck_channels=fused_bottleneck_channels, skip_channels=fused_skips, levels=levels)
        
    def forward(self, rgb, depth):
        rgb_skips, rgb_bottleneck     = self.rgb_encoder(rgb)
        depth_skips, depth_bottleneck = self.depth_encoder(depth)
        
        # Fuse bottleneck features by concatenation
        fused_bottleneck = torch.cat([rgb_bottleneck, depth_bottleneck], dim=1)
        fused_bottleneck = self.attention(fused_bottleneck)
        
        # Fuse skip connections by concatenating corresponding features
        fused_skips = [torch.cat([r, d], dim=1) for r, d in zip(rgb_skips, depth_skips)]
        
        # Decode to produce the inpainted RGB image
        out = self.decoder(fused_bottleneck, fused_skips)
        return out


In [4]:
def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    for batch in tqdm(dataloader):
        rbg_input   = batch["rgb_input"].to(device)
        depth_input = batch["depth_input"].to(device)
        target      = batch["target"].to(device)
        
        optimizer.zero_grad()

        output = model(rbg_input, depth_input)
        loss = criterion(output, target)
        loss.backward()

        optimizer.step()
        running_loss += loss.item() * rbg_input.size(0)
    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for batch in dataloader:
            rbg_input   = batch["rgb_input"].to(device)
            depth_input = batch["depth_input"].to(device)
            target      = batch["target"].to(device)

            output = model(rbg_input, depth_input)
            loss = criterion(output, target)
            
            running_loss += loss.item() * rbg_input.size(0)
    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

In [5]:
train_csv = r"C:\Users\user\Desktop\CSCE748_Computational-Photography\nyu_data\data\nyu2_train.csv"
test_csv  = r"C:\Users\user\Desktop\CSCE748_Computational-Photography\nyu_data\data\nyu2_test.csv"

full_train_dataset = DepthInpaintingDatasetCSV(train_csv, target_size=(240,320), mask_size=(32, 32), apply_mask=True)
train_size         = int(0.8 * len(full_train_dataset))
val_size           = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])  
train_loader  = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader    = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_dataset  = DepthInpaintingDatasetCSV(test_csv, target_size=(240,320), mask_size=(32, 32), apply_mask=True)
test_loader   = DataLoader(test_dataset, batch_size=16, shuffle=False)

print("Train samples:", len(train_dataset))
print("Val samples:", len(val_dataset))
print("Test samples:", len(test_dataset))

Train samples: 40550
Val samples: 10138
Test samples: 654


In [6]:
def show_sample(sample_dict, idx=0):
    rbg_input_np   = sample_dict["rgb_input"][idx].permute(1, 2, 0).cpu().numpy()
    depth_input_np = sample_dict["depth_input"][idx].squeeze(0).cpu().numpy()
    target_np      = sample_dict["target"][idx].permute(1, 2, 0).cpu().numpy()
    
    fig, axs = plt.subplots(1, 3, figsize=(15, 4))

    axs[0].imshow(rbg_input_np)
    axs[0].set_title("RGB Image")

    im = axs[1].imshow(depth_input_np, cmap="viridis")
    axs[1].set_title("Depth Image")
    axs[1].axis("off")
    fig.colorbar(im, ax=axs[1], fraction=0.046, pad=0.04)

    axs[2].imshow(target_np)
    axs[2].set_title("Target")

    plt.tight_layout()
    plt.show()

torch.Size([16, 3, 240, 320])
torch.Size([16, 1, 240, 320])
torch.Size([16, 3, 240, 320])


In [7]:
sample_batch = next(iter(train_loader))

print(sample_batch["rgb_input"].shape)            # Should be (batch_size, 4, H, W)
print(sample_batch["depth_input"].shape)          # Should be (batch_size, 3, H, W)
print(sample_batch["target"].shape)               # Should be (batch_size, 1, H, W) 

# Visualize first sample in the batch
show_sample(sample_batch, idx=0)

In [8]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = DepthEnhancedInpaintingNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.L1Loss()

num_epochs = 20
best_val_loss = float('inf')
for epoch in range(1, num_epochs+1):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss = validate(model, val_loader, criterion, device)
    print(f"Epoch {epoch}/{num_epochs} - Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "nyu_inpainting_depth.pth")
        print(f"Saved best model with Val Loss: {best_val_loss:.4f}")

cuda


c:\Users\user\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 2535/2535 [13:18<00:00,  3.17it/s]


Epoch 1/20 - Train Loss: 0.0621 | Val Loss: 0.0289
Saved best model with Val Loss: 0.0289


100%|██████████| 2535/2535 [12:58<00:00,  3.26it/s]


Epoch 2/20 - Train Loss: 0.0278 | Val Loss: 0.0235
Saved best model with Val Loss: 0.0235


100%|██████████| 2535/2535 [12:22<00:00,  3.42it/s]


Epoch 3/20 - Train Loss: 0.0226 | Val Loss: 0.0158
Saved best model with Val Loss: 0.0158


100%|██████████| 2535/2535 [12:44<00:00,  3.32it/s]


Epoch 4/20 - Train Loss: 0.0196 | Val Loss: 0.0136
Saved best model with Val Loss: 0.0136


100%|██████████| 2535/2535 [12:26<00:00,  3.40it/s]


Epoch 5/20 - Train Loss: 0.0175 | Val Loss: 0.0124
Saved best model with Val Loss: 0.0124


100%|██████████| 2535/2535 [12:27<00:00,  3.39it/s]


Epoch 6/20 - Train Loss: 0.0162 | Val Loss: 0.0130


100%|██████████| 2535/2535 [12:45<00:00,  3.31it/s]


Epoch 7/20 - Train Loss: 0.0152 | Val Loss: 0.0115
Saved best model with Val Loss: 0.0115


100%|██████████| 2535/2535 [12:23<00:00,  3.41it/s]


Epoch 8/20 - Train Loss: 0.0142 | Val Loss: 0.0100
Saved best model with Val Loss: 0.0100


100%|██████████| 2535/2535 [12:24<00:00,  3.41it/s]


Epoch 9/20 - Train Loss: 0.0138 | Val Loss: 0.0099
Saved best model with Val Loss: 0.0099


100%|██████████| 2535/2535 [12:31<00:00,  3.37it/s]


Epoch 10/20 - Train Loss: 0.0132 | Val Loss: 0.0120


100%|██████████| 2535/2535 [12:19<00:00,  3.43it/s]


Epoch 11/20 - Train Loss: 0.0136 | Val Loss: 0.0096
Saved best model with Val Loss: 0.0096


100%|██████████| 2535/2535 [12:24<00:00,  3.40it/s]


Epoch 12/20 - Train Loss: 0.0124 | Val Loss: 0.0113


100%|██████████| 2535/2535 [09:11<00:00,  4.60it/s]


Epoch 13/20 - Train Loss: 0.0122 | Val Loss: 0.0124


100%|██████████| 2535/2535 [09:11<00:00,  4.60it/s]


Epoch 14/20 - Train Loss: 0.0120 | Val Loss: 0.0072
Saved best model with Val Loss: 0.0072


100%|██████████| 2535/2535 [09:20<00:00,  4.52it/s]


Epoch 15/20 - Train Loss: 0.0116 | Val Loss: 0.0093


100%|██████████| 2535/2535 [08:32<00:00,  4.94it/s]


Epoch 16/20 - Train Loss: 0.0119 | Val Loss: 0.0091


100%|██████████| 2535/2535 [08:11<00:00,  5.16it/s]


Epoch 17/20 - Train Loss: 0.0115 | Val Loss: 0.0112


100%|██████████| 2535/2535 [08:13<00:00,  5.14it/s]


Epoch 18/20 - Train Loss: 0.0111 | Val Loss: 0.0099


100%|██████████| 2535/2535 [08:10<00:00,  5.17it/s]


Epoch 19/20 - Train Loss: 0.0109 | Val Loss: 0.0073


100%|██████████| 2535/2535 [08:08<00:00,  5.19it/s]


Epoch 20/20 - Train Loss: 0.0108 | Val Loss: 0.0068
Saved best model with Val Loss: 0.0068


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


cuda
Sample 0:
SSD between Inpainted vs GT: 32.162933
SSD between Masked vs GT: 2173.9485
Sample 1:
SSD between Inpainted vs GT: 52.26909
SSD between Masked vs GT: 2945.822
Sample 2:
SSD between Inpainted vs GT: 76.069435
SSD between Masked vs GT: 3815.762


In [9]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = DepthEnhancedInpaintingNet().to(device)
model.load_state_dict(torch.load("nyu_inpainting_depth.pth", map_location=device))
model.eval()

# Visualization: select a few samples from the test dataset.
sample_idx = 54
rows = 3
samples = [test_dataset[i] for i in range(sample_idx, sample_idx + rows)]
masked_images = torch.stack([s["rgb_input"] for s in samples], dim=0)
masked_depths = torch.stack([s["depth_input"] for s in samples], dim=0)
sample_labels = torch.stack([s["target"] for s in samples], dim=0)

inpainted_images = []
with torch.no_grad():
    for i in range(rows):
        input_masked = masked_images[i].unsqueeze(0).to(device)
        input_depth  = masked_depths[i].unsqueeze(0).to(device)
        pred = model(input_masked, input_depth)
        inpainted_images.append(pred.squeeze(0).cpu())

fig, axs = plt.subplots(nrows=rows, ncols=4, figsize=(8, 2*rows))
for i in range(rows):
    masked_np       = masked_images[i].permute(1, 2, 0).cpu().numpy()
    masked_depth_np = masked_depths[i].squeeze(0).cpu().numpy()
    inpainted_np    = inpainted_images[i].permute(1, 2, 0).cpu().numpy()
    gt_color        = sample_labels[i].permute(1, 2, 0).cpu().numpy()

    print(f"Sample {i}:")
    print("SSD between Inpainted vs GT:", np.sum((inpainted_np - gt_color)**2))
    print("SSD between Masked vs GT:", np.sum((masked_np - gt_color)**2))
    
    axs[i, 0].imshow(masked_np)
    axs[i, 0].set_title("Masked RGB Image")
    axs[i, 0].axis("off")
    
    axs[i, 1].imshow(inpainted_np)
    axs[i, 1].set_title("Inpainted")
    axs[i, 1].axis("off")
    
    axs[i, 2].imshow(gt_color)
    axs[i, 2].set_title("GT")
    axs[i, 2].axis("off")

    im = axs[i, 3].imshow(masked_depth_np, cmap="viridis")
    axs[i, 3].set_title("Masked Depth Image")
    axs[i, 3].axis("off")
    fig.colorbar(im, ax=axs[i, 3], fraction=0.046, pad=0.04)
    
plt.tight_layout()
plt.show()

In [10]:
ssd_inpainted_total = 0.0
ssd_masked_total = 0.0
num_images = 0

model.eval()
with torch.no_grad():
    for batch in tqdm(test_loader):

        rgb_input   = batch["rgb_input"].to(device)
        depth_input = batch["depth_input"].to(device)
        gt          = batch["target"].to(device)
        
        pred = model(rgb_input, depth_input)  

        pred_np   = pred.cpu().numpy()
        masked_np = rgb_input.cpu().numpy()
        gt_np     = gt.cpu().numpy()
        
        batch_size = pred_np.shape[0]
        for i in range(batch_size):
            ssd_inpainted = np.sum((pred_np[i] - gt_np[i])**2)
            ssd_masked    = np.sum((masked_np[i] - gt_np[i])**2)
            ssd_inpainted_total += ssd_inpainted
            ssd_masked_total += ssd_masked
        
        num_images += batch_size

avg_ssd_inpainted = ssd_inpainted_total / num_images
avg_ssd_masked = ssd_masked_total / num_images

print("Average SSD (Inpainted vs GT):", avg_ssd_inpainted)
print("Average SSD (Masked vs GT):", avg_ssd_masked)

100%|██████████| 41/41 [00:10<00:00,  4.03it/s]

Average SSD (Inpainted vs GT): 77.72241130443888
Average SSD (Masked vs GT): 4023.391806879539
